<a href="https://colab.research.google.com/github/BommaSreeja/sreejagit/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================
# SMART STEGANOGRAPHY SYSTEM
# AES + LSB IMPLEMENTATION
# ==========================

# Install required libraries
!pip install pycryptodome pillow

import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import hashlib
import os

# ========== AES ENCRYPTION / DECRYPTION ==========

def pad(data):
    """Pad data to be multiple of 16 bytes"""
    return data + b"\0" * (16 - len(data) % 16)

def encrypt_message(message, password):
    key = hashlib.sha256(password.encode()).digest()  # derive key
    message = pad(message.encode())
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return iv + cipher.encrypt(message)  # prepend IV for decryption

def decrypt_message(ciphertext, password):
    key = hashlib.sha256(password.encode()).digest()
    iv = ciphertext[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext[16:])
    return plaintext.rstrip(b"\0").decode(errors="ignore")

# ========== LSB STEGANOGRAPHY ==========

def encode_image(img_path, data, output_path):
    img = Image.open(img_path)
    img = img.convert("RGB")
    np_img = np.array(img)

    binary_data = ''.join(format(byte, '08b') for byte in data)
    data_len = len(binary_data)

    if data_len > np_img.size:
        raise ValueError("Data is too large to hide in this image!")

    flat_img = np_img.flatten()
    for i in range(data_len):
        flat_img[i] = (flat_img[i] & ~1) | int(binary_data[i])

    encoded_img = flat_img.reshape(np_img.shape)
    encoded_img = Image.fromarray(encoded_img.astype('uint8'), "RGB")
    encoded_img.save(output_path)
    print(f"Data hidden inside {output_path}")

def decode_image(img_path, data_size):
    img = Image.open(img_path)
    img = img.convert("RGB")
    np_img = np.array(img)

    flat_img = np_img.flatten()
    bits = [str(flat_img[i] & 1) for i in range(data_size * 8)]
    binary_data = ''.join(bits)

    data = int(binary_data, 2).to_bytes((len(binary_data) + 7) // 8, byteorder="big")
    return data

# ========== DEMO USAGE IN COLAB ==========

from google.colab import files

print("Upload cover image (PNG/JPG)...")
uploaded = files.upload()
cover_img = list(uploaded.keys())[0]

# Enter secret message
secret_message = input("Enter the secret message: ")
password = input("Enter a password for encryption: ")

# Encrypt secret message
ciphertext = encrypt_message(secret_message, password)

# Save stego image
output_img = "stego_image.png"
encode_image(cover_img, ciphertext, output_img)

# Download stego image
files.download(output_img)

# Now decode it back
print("\nDecoding process...")
extracted_data = decode_image(output_img, len(ciphertext))
decrypted_message = decrypt_message(extracted_data, password)

print("Decrypted message:", decrypted_message)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.9 MB/s eta 0:00:00
Upload cover image (PNG/JPG)...


Saving Screenshot 2025-05-27 192622.png to Screenshot 2025-05-27 192622.png
Enter the secret message: iam sreeja we need to attend the class
Enter a password for encryption: 7989


OverflowError: Python integer -2 out of bounds for uint8

In [ ]:
# ==========================
# SMART STEGANOGRAPHY SYSTEM
# LARGE IMAGE SUPPORT (AES + LSB)
# ==========================

!pip install pycryptodome pillow

import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import hashlib
from google.colab import files
import struct

# ===== AES ENCRYPTION / DECRYPTION =====

def pad(data):
    return data + b"\0" * (16 - len(data) % 16)

def encrypt_message(message, password):
    key = hashlib.sha256(password.encode()).digest()
    message = pad(message.encode())
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return iv + cipher.encrypt(message)  # prepend IV

def decrypt_message(ciphertext, password):
    key = hashlib.sha256(password.encode()).digest()
    iv = ciphertext[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext[16:])
    return plaintext.rstrip(b"\0").decode(errors="ignore")

# ===== LSB STEGANOGRAPHY WITH SIZE HEADER =====

def encode_image(img_path, data, output_path):
    img = Image.open(img_path).convert("RGB")
    np_img = np.array(img)

    # Store data length (4 bytes) + ciphertext
    data_len_bytes = struct.pack(">I", len(data))  # 4-byte big-endian length
    full_data = data_len_bytes + data

    binary_data = ''.join(format(byte, '08b') for byte in full_data)
    data_len = len(binary_data)

    if data_len > np_img.size:
        raise ValueError("❌ Data too large to hide in this image! Use a bigger image.")

    flat_img = np_img.flatten()

    for i in range(data_len):
        flat_img[i] = (flat_img[i] & 0xFE) | int(binary_data[i])

    encoded_img = flat_img.reshape(np_img.shape)
    encoded_img = Image.fromarray(encoded_img.astype('uint8'), "RGB")
    encoded_img.save(output_path)
    print(f"✅ Data hidden in {output_path} (payload: {len(data)} bytes)")

def decode_image(img_path):
    img = Image.open(img_path).convert("RGB")
    np_img = np.array(img)

    flat_img = np_img.flatten()

    # First read 32 bits (4 bytes) = length of payload
    length_bits = [str(flat_img[i] & 1) for i in range(32)]
    length = int(''.join(length_bits), 2)

    # Now read payload (length * 8 bits)
    bits = [str(flat_img[i] & 1) for i in range(32, 32 + length * 8)]
    binary_data = ''.join(bits)

    data = bytearray()
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        data.append(int(byte, 2))
    return bytes(data)

# ===== DEMO USAGE IN COLAB =====

print("📤 Upload cover image (large PNG/JPG)...")
uploaded = files.upload()
cover_img = list(uploaded.keys())[0]

secret_message = input("Enter secret message: ")
password = input("Enter password: ")

ciphertext = encrypt_message(secret_message, password)

output_img = "stego_image.png"
encode_image(cover_img, ciphertext, output_img)

files.download(output_img)

print("\n🔍 Decoding...")
extracted_data = decode_image(output_img)
decrypted_message = decrypt_message(extracted_data, password)

print("✅ Decrypted message:", decrypted_message)


📤 Upload cover image (large PNG/JPG)...


Saving Screenshot 2025-04-24 185414.png to Screenshot 2025-04-24 185414.png
Enter secret message: hey we have meet tmrw at avn college
Enter password: 7989
✅ Data hidden in stego_image.png (payload: 64 bytes)


/tmp/ipython-input-2734546590.py:57: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  encoded_img = Image.fromarray(encoded_img.astype('uint8'), "RGB")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🔍 Decoding...
✅ Decrypted message: hey we have meet tmrw at avn college


In [ ]:
# ==========================
# SMART STEGANOGRAPHY SYSTEM
# DECODING ONLY
# ==========================

!pip install pycryptodome pillow

import numpy as np
from PIL import Image
from Crypto.Cipher import AES
import hashlib
from google.colab import files
import struct

# ===== AES DECRYPTION =====
def decrypt_message(ciphertext, password):
    key = hashlib.sha256(password.encode()).digest()
    iv = ciphertext[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext[16:])
    return plaintext.rstrip(b"\0").decode(errors="ignore")

# ===== LSB DECODE FUNCTION =====
def decode_image(img_path):
    img = Image.open(img_path).convert("RGB")
    np_img = np.array(img)
    flat_img = np_img.flatten()

    # First read 32 bits = message length
    length_bits = [str(flat_img[i] & 1) for i in range(32)]
    length = int(''.join(length_bits), 2)

    # Now extract (length * 8) bits of payload
    bits = [str(flat_img[i] & 1) for i in range(32, 32 + length * 8)]
    binary_data = ''.join(bits)

    # Rebuild bytes
    data = bytearray()
    for i in range(0, len(binary_data), 8):
        data.append(int(binary_data[i:i+8], 2))
    return bytes(data)

# ===== MAIN DECODING WORKFLOW =====
print("📥 Upload stego image (the image with hidden message)...")
uploaded = files.upload()
stego_img = list(uploaded.keys())[0]

password = input("Enter the password used for encryption: ")

# Extract and decrypt
extracted_data = decode_image(stego_img)
decrypted_message = decrypt_message(extracted_data, password)

print("\n✅ Hidden message recovered:")
print(decrypted_message)


📥 Upload stego image (the image with hidden message)...


Saving stego_image.png to stego_image (1).png
Enter the password used for encryption: 7989

✅ Hidden message recovered:
hey we have meet tmrw at avn college
